In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
from snowflake.cortex import complete

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()



In [ ]:
prompt_template = """You are an expert in technical documentation and XML structuring, specializing in the S1000D standard.
Your task is to modify the provided XML document by applying the requested instructions, which may include deletion and insertion
while strictly adhering to the provided XSD schema.


The provided XML document is as follows: <XMLDOC>{the_xml}</XMLDOC>

The modifications to be applied to the XML document are the following instructions: <modification>{modification}</modification>.

The XSD document to respect and it's imports are the following: <XSD>{full_xsd}</XSD>

<output>
You must return **only the complete and well-formed XML document** after all modifications have been applied.

- The output must include the **entire XML document** from the first line (e.g., `<?xml...?>`) to the last closing tag with the entire content.
- **All unchanged parts of the document must be preserved and present.**
- All specified modifications must be included and applied in the appropriate locations.
- The XML must be syntactically valid, with all tags properly opened and closed.
- Do not add comments in the xml.
- Do not return any explanations, notes, or extra text — only the final modified XML document.
</output>

"""





continue_prompt = """
You are an expert in technical documentation and XML structuring, specializing in the S1000D standard.

You previously started modifying the provided XML document according to specific instructions and in compliance with the given XSD schema.

Here is the partial output generated so far:
<partial_output>
{previous_output}
</partial_output>

Continue generating the modified XML from where you left off, preserving all previous content and continuing seamlessly.

Do not return any explanation or additional information — only the next part of the modified XML document.
Stop exactly when you reach the end of the complete modified XML document, or when the output limit is reached.

Context:
Original XML document:
<XMLDOC>
{the_xml}
</XMLDOC>

Modifications to be applied:
<modification>
{modification}
</modification>

The XSD schema and its imports to comply with:
<XSD>
{full_xsd}
</XSD>
"""

In [ ]:
import xml.etree.ElementTree as ET
from urllib.parse import urlparse
import os

def get_contenu_from_table(xsd_name:str):
    # retourn le contenu d'un xsd
    df = session.table("XSD_FILES_2")
    
    filtered_df = df.filter(df["nom_xsd"] == xsd_name).select("contenu")
    
    # Collect result and extract value
    rows = filtered_df.collect()
    
    if rows:
        contenu_value = rows[0]["CONTENU"]  # Case-sensitive key
        return contenu_value
        #print("Contenu for file_xsd = 'proced':", contenu_value)
    else:
        return ""


def extract_imported_xsds(xsd_string):
    #extract les imports xsd
    try:
        root = ET.fromstring(xsd_string)
        imports = []
        imports_w_extension = []

        # Parcours des éléments <xs:import> pour extraire les schémas importés
        for elem in root.iter():
            if elem.tag.endswith('import'):
                schema_location = elem.attrib.get('schemaLocation')
                if schema_location:
                    # Extraction du nom du fichier sans l'extension .xsd
                    filename = os.path.basename(schema_location.strip())
                    name_without_extension = os.path.splitext(filename)[0]
                    imports_w_extension.append(filename)
                    imports.append(name_without_extension)

        return imports, imports_w_extension
    except ET.ParseError as e:
        print(f"Erreur lors de l'analyse du XSD : {e}")
        return []
        

def extract_xsd_name(xml_string):
    # extract le xsd du xml
    try:
        root = ET.fromstring(xml_string)
        xsi_ns = "http://www.w3.org/2001/XMLSchema-instance"

        # Get noNamespaceSchemaLocation
        no_ns_location = root.attrib.get(f"{{{xsi_ns}}}noNamespaceSchemaLocation")
        schema_url = None

        if no_ns_location:
            schema_url = no_ns_location.strip()
        else:
            # Fallback to schemaLocation if needed
            schema_location = root.attrib.get(f"{{{xsi_ns}}}schemaLocation")
            if schema_location:
                parts = schema_location.strip().split()
                if parts:
                    schema_url = parts[-1]  # Assume last is the schema URL

        if schema_url:
            filename = os.path.basename(urlparse(schema_url).path)
            return os.path.splitext(filename)[0]  # Return filename without .xsd

        return None

    except ET.ParseError as e:
        print(f"XML parsing error: {e}")
        return None


def get_all_xsd_content(xml):

    xsd_prin_name = extract_xsd_name(xml)
    xsd_princ_content = get_contenu_from_table(xsd_prin_name)
    import_names, import_w_extension = extract_imported_xsds(xsd_princ_content)
    
    importes_content = []
    importes_map = {}
    #for i in import_names:
    #    importes_content.append(get_contenu_from_table(i))
    for i in range(len(import_names)):
        icontent = get_contenu_from_table(import_names[i])
        importes_content.append(icontent)
        importes_map[import_w_extension[i]] = icontent

    full_xsd = "This is the main xsd:\n"
    full_xsd = full_xsd + xsd_princ_content + "\n\n"
    
    for i in importes_content:
        full_xsd = full_xsd + "One of the imports of the xsd : \n" + i + "\n"
    return full_xsd, xsd_princ_content, importes_map

#full_xsd,main_xsd, importes_map = get_all_xsd_content(the_xml)

In [ ]:
from snowflake.snowpark.functions import col

df_case = session.table("TEST_CASE2")

# 3. Sort the table to get the "last" row — you need to define what "last" means (usually based on a timestamp or ID)
# Replace 'your_column' with the appropriate column to sort by
#rows = session.table("TEST_CASE2").collect()

In [ ]:
query = """
SELECT SNOWFLAKE.CORTEX.COMPLETE(
    'claude-3-5-sonnet',
    TO_VARIANT(ARRAY_CONSTRUCT(
        OBJECT_CONSTRUCT(
            'role', 'user',
            'content', ?
        )
    )),
    OBJECT_CONSTRUCT('max_tokens', 8192)
) AS result;
"""

In [ ]:
def clean_xml_string_simple(s):
    return s.replace("```xml", "").replace("```", "").strip()

In [ ]:
import json

rows = df_case.collect()

new_data = []
# Loop through each row
for row in rows:
    original = row["XML"]
    instruction = row["INSTRUCTION"]
    xml_modif = row["XML_MODIF"]

    final_xml = None
    full_xsd, main_xsd, importes_map = get_all_xsd_content(original)
    prompt_xml = prompt_template.format(the_xml=original, modification=instruction, full_xsd=full_xsd)
    
    df = session.sql(query, params=[prompt_xml])
    result = df.collect()[0]['RESULT']
    response = json.loads(result)  # convertir en dictionnaire Python
    edited_xml = response['choices'][0]['messages']
    nb_token = response['usage']["completion_tokens"]

    continue_xml = None
            #continue part
    if nb_token > 4080:
        
        edited_xml = edited_xml.rsplit('\n', 1)[0]
        next_prompt = continue_prompt.format(previous_output=edited_xml, the_xml=original, modification=instruction, full_xsd=full_xsd)
        df = session.sql(query, params=[next_prompt])
        result = df.collect()[0]['RESULT']
        response = json.loads(result)  # convertir en dictionnaire Python
        continue_xml = response['choices'][0]['messages']
        continue_xml = clean_xml_string_simple(continue_xml)
        nb_token = response['usage']["completion_tokens"]
        
    if continue_xml != None:
                final_xml = edited_xml + "\n" + continue_xml
    else: 
        final_xml = edited_xml

    new_data.append({
        "XML": original,
        "XML_MODIF": xml_modif,
        "INSTRUCTIONS": instruction,
        "PREDICTED_XML": final_xml
    })

df_with_predictions = session.create_dataframe(pd.DataFrame(new_data))

# Step 5: Save as a new table
df_with_predictions.write.mode("overwrite").save_as_table("TEST_CASE2_PREDICTED")

In [ ]:
df_predicted = session.table("TEST_CASE2_PREDICTED")
df_predicted